In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
import calendar
import json
import csv
import numpy as np
from selenium import webdriver
import time
import math
import os
import re
import datetime
from selenium.webdriver.chrome.options import Options
import warnings
from selenium.webdriver.support.ui import Select

# Competitions

In this Notebook we will generate competitions. Since new gas stations do not arrive every second we can use this dataframe throughout our analysis, assuming that it is fix, so we will not need to rerun this script again. In the end we want a dataframe witht the following columns:

| City | 1 km competition | 1 km competition | 5 km competition | 10 km competition | 20 km competition | 30 km competition | 40 km competition | 

We can include the number of competitions, which will only be the lenght of these lists.

In [15]:
warnings.filterwarnings('ignore', category = DeprecationWarning)
chrome_options = Options()
chrome_options.add_argument('--headless')

driver = webdriver.Chrome('/home/csaba/Downloads/chromedriver', options=chrome_options)
driver.get('https://holtankoljak.net/')
time.sleep(4)

#Click on 'OK tovább pop-up':
driver.find_element('xpath','/html/body/div[1]/div/div/div/div[2]/div/button[2]').click()
time.sleep(5)

mutasd_buttons = driver.find_elements('xpath', "//input[@type='submit' and @value='Mutasd']")
for button in mutasd_buttons:
    if button.is_enabled() and button.is_displayed():
        mutasd_button = button
        break

time.sleep(3)

toggles = driver.find_elements('xpath', '/html/body/div[2]/div[2]/div/div[1]/form[1]/div[1]/select')
fuel_toggle = Select(toggles[0])
fuel_toggle.select_by_index(1)

city_toggle = Select(toggles[1])
radius_toggle = Select(toggles[2])

city_toggle_options = city_toggle.options
radius_toggle_options = radius_toggle.options

competitions = pd.DataFrame(columns = ['City', '1km', 'Number of competition 1km','5km', 'Number of competition 5km','10km', 'Number of competition 10km','20km', 'Number of competition 20km','30km', 'Number of competition 30km','40km', 'Number of competition 40km'])

for index in range(1,len(city_toggle_options)):
    
    #we need to obtain in each for loop our buttons and toggle bars
    
    time.sleep(3)
    mutasd_buttons = driver.find_elements('xpath', "//input[@type='submit' and @value='Mutasd']")
    for button in mutasd_buttons:
        if button.is_enabled() and button.is_displayed():
            mutasd_button = button
            break
    time.sleep(3)
    
    toggles = driver.find_elements(By.TAG_NAME, 'select')

    city_toggle = Select(toggles[1])  #obtain the tuggles
    city_toggle.select_by_index(index)
    city = city_toggle.first_selected_option.text #this is the city
    
    time.sleep(3)
    
    competition_data_row = []
    competition_data_row.append(city)
    
    for radius in range(1, len(radius_toggle_options)):
        
        time.sleep(5)
        mutasd_buttons = driver.find_elements('xpath', "//input[@type='submit' and @value='Mutasd']")
        for button in mutasd_buttons:
            if button.is_enabled() and button.is_displayed():
                mutasd_button = button
                break
        time.sleep(3)
        
        toggles = driver.find_elements(By.TAG_NAME, 'select')
        radius_toggle = Select(toggles[2])  #obtain the tuggles
        radius_toggle.select_by_index(radius)
        radius = radius_toggle.first_selected_option.text

        mutasd_button.click()
        time.sleep(5)
        #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        
        rows = driver.find_elements('xpath', '/html/body/section/div[1]/div[2]/div/div[2]/div/div/table/tbody/tr[2]/td/div[1]/table/tbody/tr')
        competitions_in_radius = []
                
        for row in rows:
            
            try:
                competitions_in_radius.append(row.find_element('xpath', './/td[3]').text)
            except:
                pass
        
        competition_data_row.append(competitions_in_radius)
        competition_data_row.append(len(competitions_in_radius))

        driver.back()
        
    competitions.loc[len(competitions)] = competition_data_row

driver.quit()
competitions.to_csv('competitions.csv', index=False, sep = '\t')

   City                    1km  Number of competition 1km  \
0  Abda  [Abda, Bécsi út 186.]                          1   

                                        5km  Number of competition 5km  \
0  [Abda, Bécsi út 186., Abda, Bécsi u. 1.]                          2   

                                                10km  \
0  [Abda, Bécsi út 186., Abda, Bécsi u. 1., Győr,...   

   Number of competition 10km  \
0                          24   

                                                20km  \
0  [Abda, Bécsi út 186., Abda, Bécsi u. 1., Győr,...   

   Number of competition 20km  \
0                          37   

                                                30km  \
0  [Abda, Bécsi út 186., Abda, Bécsi u. 1., Győr,...   

   Number of competition 30km  \
0                          53   

                                                40km  \
0  [Abda, Bécsi út 186., Abda, Bécsi u. 1., Győr,...   

   Number of competition 40km  
0                          68  


KeyboardInterrupt: 

# External attributes

In [2]:
path = '/home/csaba/Desktop/Gas Competition Analysis/Area_Data'
os.chdir(path)
os.listdir()

['lakasallomany.csv',
 'Kiskereskedelmiuzletek.csv',
 '.ipynb_checkpoints',
 'munkanelkuliek1564.csv',
 'jarandosag.csv',
 'szjakepezojovegylakos.csv',
 'kulfoldreingazok.csv',
 'ivoviz.csv',
 'foglalkoztatottak1564.csv',
 'ezerlakosadozo.csv',
 'szemelygepkocsiezerlakos.csv',
 'gazdasagszam.csv',
 'ingazasikulonbozet.csv',
 'szjaalapperfo.csv',
 'helybenfoglalkoztatottak.csv']

In [3]:
external_attributes = pd.DataFrame()

# Iterate over the file names
for filename in os.listdir():
    if filename.endswith('.csv') and filename != '.ipynb_checkpoints':
        # Read the CSV file, skip the first row, and select the 'TELEP_NEV' and 'VALUE' columns
        df = pd.read_csv(filename, skiprows=1, usecols=['TELEP_NEV', 'VALUE'])

        # Rename the 'VALUE' column to the filename without the '.csv' extension
        df = df.rename(columns={'VALUE': os.path.splitext(filename)[0]})

        # Merge the resulting data frame with the previous result on the 'TELEP_NEV' column
        if external_attributes.empty:
            external_attributes = df
        else:
            external_attributes = pd.merge(external_attributes, df, on='TELEP_NEV', how='outer')
            
external_attributes.columns = ['City_for_merge' if x=='TELEP_NEV' else x for x in external_attributes.columns]
external_attributes

,City_for_merge,lakasallomany,Kiskereskedelmiuzletek,munkanelkuliek1564,jarandosag,szjakepezojovegylakos,kulfoldreingazok,ivoviz,foglalkoztatottak1564,ezerlakosadozo,szemelygepkocsiezerlakos,gazdasagszam,ingazasikulonbozet,szjaalapperfo,helybenfoglalkoztatottak
0,Budapest,549,146.0,7.5,23.5,2508,1.8,100.6,63.2,508,411.8,2157.0,NaN,4936,NaN
1,Magyarpolány,408,47.0,4.4,26.8,1881,3.4,97.9,64.2,502,433.0,48.0,6.0,3748,23.3
2,Farkasgyepű,466,28.0,2.9,31.4,2099,1.6,98.8,64.9,569,514.1,17.0,4.0,3688,62.5
3,Öcs,609,57.0,6.1,29.3,1172,NaN,95.3,54.2,484,431.0,12.0,50.0,2423,10.0
4,Nyirád,419,50.0,6.9,27.9,1562,1.6,100.0,58.5,514,411.7,58.0,3.0,3039,30.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3150,Pénzesgyőr,451,31.0,12.3,27.0,1624,NaN,98.0,53.1,555,365.0,26.0,43.0,2928,30.4
3151,Bakonybél,471,90.0,9.2,31.2,1704,1.7,99.8,58.6,500,388.8,30.0,3.0,3408,13.6
3152,Bakonyoszlop,478,92.0,11.6,26.4,1531,NaN,96.2,51.1,475,349.4,20.0,6.0,3225,56.8
3153,Porva,371,65.0,11.0,18.4,1839,NaN,100.6,53.8,541,460.0,30.0,15.0,3398,47.1


In [4]:
path = '/home/csaba/Desktop/Gas Competition Analysis'
os.chdir(path)
external_attributes.to_csv('external_attributes.csv', index = False)